In [ ]:
%%capture
!pip install selenium
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
import sys
import csv
from selenium import webdriver
import time
from bs4 import BeautifulSoup

from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
driver = webdriver.Chrome(options=chrome_options)


In [ ]:
path_to_file = "output.csv"

csvFile = open(path_to_file, 'a', encoding="utf-8")
csvWriter = csv.writer(csvFile)

all_rows = []

all_urls = [  "https://forum-exemple.com/topic/qsdfg",
              "https://forum-exemple.com/topic/dfskjndnk"
]

for url in all_urls:
    try:
        driver.get(url)
    except Exception as e:
        print(f"Error {url}: {e}")
        continue

    time.sleep(3)

    while True:
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        print(driver.current_url)

        comment_elements = soup.find_all('balise', class_='balise')

        for comment_element in comment_elements:

            h3_element = comment_element.select_one('balise')
            if h3_element:
                pseudo_text = h3_element.get_text(strip=True)
            else:
                a_element = comment_element.find('a', class_='balise')
                if a_element:
                    pseudo_text = a_element.get_text(strip=True)
                else:
                    strong_element = comment_element.find('balise')
                    if strong_element:
                        pseudo_text = strong_element.get_text(strip=True)
                    else:
                        pseudo_text = ""


            date_element = comment_element.find('time')
            date_text = date_element['datetime'] if date_element else ""

            # Extraction du contenu du commentaire
            content_element = comment_element.find('div', class_='balise')
            sub_element_text = '\n'.join([p.get_text(strip=True) for p in content_element.find_all('p')]) if content_element else ""

            # Ajout des données extraites à la liste
            all_rows.append([url, pseudo_text, date_text, sub_element_text])

        # Navigation vers la page suivante
        try:
          next_page = driver.find_element(By.XPATH, "balise")
          a_element = next_page.find_element(By.TAG_NAME, 'a')
          href_value = a_element.get_attribute('href')
          href_value = href_value.replace('#comments', '')
          driver.get(href_value)
          time.sleep(3)
        except NoSuchElementException:
          print("No more pages for", url)
          break

# Écriture des données dans le fichier CSV
    for row in all_rows:
      csvWriter.writerow(row)

# Fermeture du fichier CSV et du WebDriver
csvFile.close()
driver.close()


In [ ]:
#csvFile.close()